In [1]:
# take in live currency trading rates from Oanda, look for price discrepencies 

In [1]:
import os
import urllib
import datetime
from bs4 import BeautifulSoup
import time
import pandas as pd 
import json
import numpy as np
import requests

In [2]:
from typing import Tuple, List
from math import log

currencies = [] 
df_c = [] 


def negate_logarithm_convertor(graph: Tuple[Tuple[float]]) -> List[List[float]]:
    ''' log of each rate in graph and negate it'''
    result = [] 
    for i, row in enumerate(graph):
        result.append([])
        for edge in row:
            try:
                result[i].append(-log(edge))
            except:
                result[i].append(None)
                
    return result


def arbitrage(currency_tuple: tuple, rates_matrix: Tuple[Tuple[float, ...]]):
    ''' Calculates arbitrage situations and prints out the details of this calculations'''
    global currencies 
    global df_c 
    
    trans_graph = negate_logarithm_convertor(rates_matrix)

    # Pick any source vertex -- we can run Bellman-Ford from any vertex and get the right result

    source = 0
    n = len(trans_graph)
    min_dist = [float('inf')] * n

    pre = [-1] * n
    
    min_dist[source] = source

    # 'Relax edges |V-1| times'
    for _ in range(n-1):
        for source_curr in range(n):
            for dest_curr in range(n):
                try:
                    if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                        min_dist[dest_curr] = min_dist[source_curr] + trans_graph[source_curr][dest_curr]
                        pre[dest_curr] = source_curr
                except:
                    pass

    # if we can still relax edges, then we have a negative cycle
    for source_curr in range(n):
        for dest_curr in range(n):
            try:
                
                if min_dist[dest_curr] > min_dist[source_curr] + trans_graph[source_curr][dest_curr]:
                    # negative cycle exists, and use the predecessor chain to print the cycle
                    print_cycle = [dest_curr, source_curr]
                    # Start from the source and go backwards until you see the source vertex again or any vertex that already exists in print_cycle array
                    while pre[source_curr] not in  print_cycle:
                        print_cycle.append(pre[source_curr])
                        source_curr = pre[source_curr]
                    print_cycle.append(pre[source_curr])
                    
                    returned = "_".join([currencies[p] for p in print_cycle[::-1]])

                    counter = 0 
                    yie1d = 1
                    for char in returned:
                        if char == '_':
                            yie1d = yie1d * df_c[returned[counter - 3: counter]][returned[counter + 1: counter + 4]]
                        counter += 1
                        
                    print("Arbitrage Opportunity: ")
                    print(returned)
                    print("Yield: ")
                    print(yie1d)
                    
            except:
                pass
            
            

In [3]:
def soup():
    
    global currencies
    global df_c 
    

    boolean = True
    while True:
        if boolean:
            url = "https://www1.oanda.com/currency/live-exchange-rates/#USD"
            boolean = False 
        else:
            boolean = True
            url = "https://www1.oanda.com/currency/live-exchange-rates/#GBP"
        
        r = requests.get(url)
        soup = BeautifulSoup(r.text, "xml")
        if url[-3:] != 'USD':
            continue
        
        content = soup.find("div", {"id": "menu_content"})
        
        trades = []
        currencies = [] 
        for item in content.find_all(class_=["rate_row"]):
            trade_names = item.find_all(class_='inline title left')
            for name in trade_names:
                nam = name.text.replace('/', '_').replace('\n', '')
                trades.append(nam)
                if nam[:3] not in currencies:
                    currencies.append(nam[:3])
                
            prices = item.find_all(class_='inline value right')
            
        len_currencies = len(currencies)

        all_scripts = soup.find_all('script')
        # print(all_scripts[11:15])
        data = json.loads(all_scripts[13].get_text()[152:len(all_scripts[13].get_text()) - 6])
        
        bid, ask, extras, spread = [], [], [], [] 
        for trade in trades: 
            bid.append(data[trade]['bid'])
            ask.append(data[trade]['ask'])
            extras.append(data[trade]['extras'])
            spread.append(data[trade]['spread'])

        print(ask)

        df = pd.DataFrame(list(zip(trades, bid, ask, spread, extras)), columns =['Pair', 'Bid', 'Ask', 'Spread', 'Extras'])
        rates = []
        for i in range(len_currencies):
            rates.append(np.zeros(len_currencies))
        df_c = pd.DataFrame(rates, index = currencies, columns = currencies)
        
        for i, trade in enumerate(trades):
            df_c[trade[:3]][trade[4:]] = ask[i]
            for j in range(len_currencies):
                if i == j:
                    df_c.iloc[i][j] = 1.0
                try:
                    if df_c.iloc[i][j] == 0.0:

                        df_c.iloc[i][j] = None
                except:
                    pass
        
        new_rates = df_c.values
        arbitrage(currencies, new_rates)
            
        time.sleep(5)

soup()

['1.17802', '0.84888', '1.30782', '0.76463', '1.31977', '0.75771', '0.91257', '1.09581', '106.000', '0.00943', '0.90083', '1.11009', '1.07490', '0.93032', '0.71542', '1.39778', '124.863', '0.00801', '138.625', '0.00721', '1.64675', '0.60726', '26.10019', '0.03831', '350.941', '0.00285', '1.80440', '0.55420', '10.36374', '0.09649', '1.61718', '0.61836', '1.55458', '0.64326', '7.44657', '0.13429', '10.65646', '0.09384', '4.40993', '0.22676', '8.64997', '0.11561', '20.17516', '0.04957', '6.91924', '0.14452', '6.32187', '0.15818', '297.927', '0.00336', '22.01655', '0.04542', '3.74381', '0.26711', '8.79841', '0.11366', '31.614', '0.03163', '17.12633', '0.05839', '22.15554', '0.04514', '7.75054', '0.12902', '74.877', '0.01336', '9.04662', '0.11054', '3.75157', '0.26656', '1.37279', '0.72844', '7.34653', '0.13612', '1.82828', '0.54696', '1.19342', '0.83793', '22.39816', '0.04465', '1.79536', '0.55699', '75.831', '0.01319', '0.98212', '1.01821', '80.326', '0.01245', '116.171', '0.00861', '0.86

KeyboardInterrupt: 